In [2]:
import boto3
from botocore.client import Config
from dotenv import load_dotenv
import os

# Get credentials from environment variables
load_dotenv("config.env")
minio_access_key = os.environ.get("minio_access_key")
minio_secret_key = os.environ.get("minio_secret_key")
s3_access_key = os.environ.get("ACCESS_KEY_ID")
s3_secret_key = os.environ.get("SECRET_ACCESS_KEY")

# Initialize S3 resource for CDSE
session = boto3.session.Session()
s3 = session.resource(
    's3',
    endpoint_url='https://eodata.dataspace.copernicus.eu',
    aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key,
    region_name='default'
)

# Initialize S3 resource for MinIO
minio_s3 = boto3.resource(
    's3',
    endpoint_url='https://pangeo-eosc-minioapi.vm.fedcloud.eu',  # MinIO API endpoint:contentReference[oaicite:4]{index=4}
    aws_access_key_id=minio_access_key,
    aws_secret_access_key=minio_secret_key,
    region_name='default',  # or any region; required by boto3 but not used by MinIO
    config=Config(signature_version='s3v4')  # Use V4 signatures for MinIO:contentReference[oaicite:5]{index=5}
)
source_bucket = s3.Bucket("eodata")               # source bucket (Copernicus)
dest_bucket   = minio_s3.Bucket("okihle-s2l1b")   # destination bucket (MinIO)

In [ ]:
prefix = "Sentinel-2/MSI/MSI_L1B_GR/2025/07/24/"
for obj in source_bucket.objects.filter(Prefix=prefix):
    if obj.key.endswith("/"):
        continue  # skip folder placeholders
    print(f"Copying {obj.key} ...")
    # Stream from source and upload to destination
    source_response = source_bucket.Object(obj.key).get()
    dest_bucket.upload_fileobj(source_response['Body'], obj.key)

Copying Sentinel-2/MSI/MSI_L1B_GR/2025/07/24/S2A_OPER_MSI_L1B_GR_2APS_20250724T011624_S20250724T000002_D02_N05.11.tar ...
Copying Sentinel-2/MSI/MSI_L1B_GR/2025/07/24/S2A_OPER_MSI_L1B_GR_2APS_20250724T011624_S20250724T000002_D12_N05.11.tar ...
Copying Sentinel-2/MSI/MSI_L1B_GR/2025/07/24/S2A_OPER_MSI_L1B_GR_2APS_20250724T011624_S20250724T000002_D04_N05.11.tar ...
Copying Sentinel-2/MSI/MSI_L1B_GR/2025/07/24/S2A_OPER_MSI_L1B_GR_2APS_20250724T011624_S20250724T000002_D10_N05.11.tar ...
Copying Sentinel-2/MSI/MSI_L1B_GR/2025/07/24/S2A_OPER_MSI_L1B_GR_2APS_20250724T011624_S20250724T000002_D06_N05.11.tar ...
Copying Sentinel-2/MSI/MSI_L1B_GR/2025/07/24/S2A_OPER_MSI_L1B_GR_2APS_20250724T011624_S20250724T000002_D08_N05.11.tar ...
Copying Sentinel-2/MSI/MSI_L1B_GR/2025/07/24/S2A_OPER_MSI_L1B_GR_2APS_20250724T011624_S20250724T000002_D07_N05.11.tar ...
Copying Sentinel-2/MSI/MSI_L1B_GR/2025/07/24/S2A_OPER_MSI_L1B_GR_2APS_20250724T011624_S20250724T000002_D05_N05.11.tar ...
Copying Sentinel-2/MSI/M